In [1]:
import sys
sys.path.insert(0, '/home/bibek/projects/DEEPL')
import numpy as np
import pandas as pd
import csv
import json

In [2]:
from helpers.deep import get_sector_excerpt, process_deep_entries_data
data = process_deep_entries_data('nlp/nlp_out.csv')

In [3]:
import random
random.shuffle(data)

SAMPLE_SIZE = 100
sectors_samples = {}  # to hold samples of all sectors
samples_counts = {}  # to hold samples count of sectors
for excerpt, label in data:
    if not samples_counts.get(label):
        samples_counts[label] = 0
    if samples_counts[label] < SAMPLE_SIZE:
        sectors_samples[label] = sectors_samples.get(label, []) + [excerpt]
        samples_counts[label] += 1


In [4]:
def get_confidence_interval_discrete(percent, true, total):
    factor = 2 if percent == 95 else 1.5 # TODO: add for other confidence levels
    adjusted_true = true + 2
    adjusted_total = total + 4
    prob = adjusted_true/float(adjusted_total)
    print(prob)
    var = prob * (1-prob)
    err = var/adjusted_total
    sqrt = err**0.5
    error_margin = sqrt*factor
    return (prob-error_margin, prob+error_margin)

In [8]:
# now get the model
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'deepl.settings'
import django
django.setup()
from classifier.models import ClassifierModel

import pickle

model = ClassifierModel.objects.last()
for model in ClassifierModel.objects.all():
    classifier = pickle.loads(model.data)

    confidences = {}
    for label, entries in sectors_samples.items():
        total_classified = len(entries)
        correct_classified = 0
        for entry in entries:
            classified = classifier.classify(classifier.preprocess(entry))
            if classified == label:
                correct_classified += 1
        confidence95 = get_confidence_interval_discrete(95, correct_classified, total_classified)
        confidences[label] = confidence95
        model.metadata['confidences_95'] = confidences
print(confidences)

0.8365384615384616
0.7115384615384616
0.6538461538461539
0.8365384615384616
0.6634615384615384
0.7019230769230769
0.5096153846153846
0.25
0.8365384615384616
0.5865384615384616
0.6057692307692307
0.5480769230769231
{'Protection': (0.7640174131600967, 0.9090595099168264), 'Food': (0.62268865757133, 0.8003882655055932), 'Shelter': (0.5605452838389262, 0.7471470238533815), 'Health': (0.7640174131600967, 0.9090595099168264), 'Nutrition': (0.5707916683593254, 0.7561314085637515), 'WASH': (0.612216917498603, 0.7916292363475508), 'Livelihood': (0.4115754507621331, 0.6076553184686361), 'NFI': (0.16507922243915532, 0.3349207775608447), 'Education': (0.7640174131600967, 0.9090595099168264), 'Agriculture': (0.48996025594747306, 0.6831166671294501), 'Logistic': (0.5099302492544635, 0.701608212283998), 'Cross': (0.4504732091131731, 0.6456806370406731)}


In [7]:
model.metadata['confidences'] = confidences
model.save()